In [ ]:
import time
from pynq import Overlay
import pynq.lib.dma
from pynq import Xlnk
import numpy as np
from pynq import MMIO
import random

In [ ]:
ol = Overlay('/home/xilinx/jupyter_notebooks/sadd/sadd.bit') # check this path
ol.download() # this downloads your bitstream into FPGA
dma1 = ol.streamAdd.sadd_dma1 # first DMA. Note that we had to access the hierarchy before accessing the DMA
dma2 = ol.streamAdd.sadd_dma2 # second DMA
sadd_ip = MMIO(0x40000000, 0x10000) # we got this address from
xlnk = Xlnk()

In [ ]:
length = 8

in_buffer1 = xlnk.cma_array(shape=(length,), dtype=np.int32) # input buffer 1
in_buffer2 = xlnk.cma_array(shape=(length,), dtype=np.int32) # input buffer 2
out_buffer = xlnk.cma_array(shape=(length,), dtype=np.int32) # output buffer

samples = random.sample(range(0, length), length)
np.copyto(in_buffer1, samples)
samples = random.sample(range(0, length), length)
np.copyto(in_buffer2, samples)

sadd_ip.write(0x10, length) # we got this address from Vivado source. Since we didn't do port=return, and we set a constant for ap_start, we only have to write length.
t_start = time.time()
dma1.sendchannel.transfer(in_buffer1)
dma2.sendchannel.transfer(in_buffer2)
dma1.recvchannel.transfer(out_buffer)
dma1.sendchannel.wait()
dma2.sendchannel.wait()
dma1.recvchannel.wait()
t_stop = time.time()
in_buffer1.close()
in_buffer2.close()
out_buffer.close()
print('Hardware execution time: ', t_stop-t_start)
for i in range(0, length):
    print('{}+{} = {}'.format(in_buffer1[i], in_buffer2[i], out_buffer[i]))